In [83]:
import models
import utils
import data_processing
from constants import (
    VALUE_COLUMN,
    UNIT_RATE_COLUMN,
    QUANTITY_COLUMN,
    GROSS_WEIGHT_COLUMN,
    SPIKES_WINDOW_SIZE,
    SHIP_COUNT_COLUMN,
    PORT_COUNT_COLUMN,
    FILL_METHOD,
    TARGET_COLUMN
)
import pandas as pd
import constants
import sys
from datetime import datetime
from build_data import get_data
from sklearn.model_selection import train_test_split

In [84]:
COMMODITY = "magnesium"

VOLZA_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}.csv"
PRICE_FILE_PATH = f"../volza/{COMMODITY}/{COMMODITY}_prices.csv"

# Get the data
data = get_data(VOLZA_FILE_PATH, PRICE_FILE_PATH)

/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brent_df.rename(columns={"Value": "Brent Oil Value"}, inplace=True)
/Users/harshdeepsingh/ASU/Lab_V2/GSN/onr_price_prediction/analysis/build_data.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wti_df.rename(columns={"Value": "WTI Oil Value"}, inplace=True)


In [85]:
#Volza Only Features
aggregated_df = data
VOLZA_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN]
X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

6/6 [==============================] - 0s 622us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.845714,0.868263,0.966667,0.914826,0.375000,0.12,0.181818
1,LSTM,200 layers,0.811429,0.867925,0.920000,0.893204,0.250000,0.16,0.195122
2,LSTM,100 layers,0.851429,0.869048,0.973333,0.918239,0.428571,0.12,0.187500
3,LSTM,50 layers,0.834286,0.862275,0.960000,0.908517,0.250000,0.08,0.121212
4,CNN with Attention,"32 filters, kernel size 7",0.845714,0.868263,0.966667,0.914826,0.375000,0.12,0.181818
5,CNN with Attention,"32 filters, kernel size 5",0.840000,0.863095,0.966667,0.911950,0.285714,0.08,0.125000
6,CNN with Attention,"32 filters, kernel size 3",0.845714,0.863905,0.973333,0.915361,0.333333,0.08,0.129032
7,CNN with Attention,"64 filters, kernel size 7",0.828571,0.861446,0.953333,0.905063,0.222222,0.08,0.117647
8,CNN with Attention,"64 filters, kernel size 5",0.845714,0.863905,0.973333,0.915361,0.333333,0.08,0.129032
9,CNN with Attention,"64 filters, kernel size 3",0.822857,0.860606,0.946667,0.901587,0.200000,0.08,0.114286


In [86]:
#Volza and AIS Features
aggregated_df = data
VOLZA_AIS_FEATURES = [VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN ]
aggregated_df[VOLZA_AIS_FEATURES] = aggregated_df[VOLZA_AIS_FEATURES].fillna(0)

X, y = data_processing.prepare_features_and_target(aggregated_df, VOLZA_AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

output_file_path = f'{COMMODITY}_results_volza_ais.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

6/6 [==============================] - 0s 742us/step


,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.851429,0.864706,0.980000,0.918750,0.400000,0.08,0.133333
1,LSTM,200 layers,0.822857,0.865031,0.940000,0.900958,0.250000,0.12,0.162162
2,LSTM,100 layers,0.834286,0.862275,0.960000,0.908517,0.250000,0.08,0.121212
3,LSTM,50 layers,0.828571,0.861446,0.953333,0.905063,0.222222,0.08,0.117647
4,CNN with Attention,"32 filters, kernel size 7",0.834286,0.862275,0.960000,0.908517,0.250000,0.08,0.121212
5,CNN with Attention,"32 filters, kernel size 5",0.800000,0.870968,0.900000,0.885246,0.250000,0.20,0.222222
6,CNN with Attention,"32 filters, kernel size 3",0.811429,0.863354,0.926667,0.893891,0.214286,0.12,0.153846
7,CNN with Attention,"64 filters, kernel size 7",0.742857,0.857143,0.840000,0.848485,0.142857,0.16,0.150943
8,CNN with Attention,"64 filters, kernel size 5",0.771429,0.852564,0.886667,0.869281,0.105263,0.08,0.090909
9,CNN with Attention,"64 filters, kernel size 3",0.811429,0.863354,0.926667,0.893891,0.214286,0.12,0.153846


In [87]:
#AIS Features
aggregated_df = data
AIS_FEATURES = [SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN]
# aggregated_df = aggregated_df.dropna(subset=[SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN])
aggregated_df[AIS_FEATURES] = aggregated_df[AIS_FEATURES].fillna(FILL_METHOD)
X, y = data_processing.prepare_features_and_target(aggregated_df, AIS_FEATURES, 'spikes')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train, X_test = data_processing.scale_features(X_train, X_test)

X_train, y_train = data_processing.create_sequences(X_train, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test, y_test, SPIKES_WINDOW_SIZE)

# print(y_test)
output_file_path = f'{COMMODITY}_results_ais_only.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

6/6 [==============================] - 0s 5ms/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

6/6 [==============================] - 0s 4ms/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

6/6 [==============================] - 0s 3ms/step


...

6/6 [==============================] - 0s 577us/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

6/6 [==============================] - 0s 740us/step


/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harshdeepsingh/miniconda/envs/lab_v2/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
1,LSTM,200 layers,0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
2,LSTM,100 layers,0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
3,LSTM,50 layers,0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
4,CNN with Attention,"32 filters, kernel size 7",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
5,CNN with Attention,"32 filters, kernel size 5",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
6,CNN with Attention,"32 filters, kernel size 3",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
7,CNN with Attention,"64 filters, kernel size 7",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
8,CNN with Attention,"64 filters, kernel size 5",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0
9,CNN with Attention,"64 filters, kernel size 3",0.857143,0.857143,1.000000,0.923077,0.0,0.0,0.0


In [89]:
#AIS and Volza with Price ARIMA
import pmdarima as pm

aggregated_df = data

columns_of_interest = ['Price', 'spikes']  # Add other columns as necessary

aggregated_df = aggregated_df.dropna(subset=columns_of_interest)
# Fit an Auto ARIMA model to the 'Price' series
model = pm.auto_arima(aggregated_df['Price'], seasonal=True, m=12, suppress_warnings=True, stepwise=True, error_action='ignore')

# Forecast the series using the model (in-sample prediction)
forecast = model.predict_in_sample()

# Calculate residuals (difference between actual and forecasted values)
residuals = aggregated_df[TARGET_COLUMN] - forecast

# Append residuals to DataFrame as a new feature (using residuals as a way to detect spike / anomaly)
aggregated_df = aggregated_df.copy()
aggregated_df['ARIMA_Residuals'] = residuals

# Prepare features and target
FEATURE_COLUMNS = [TARGET_COLUMN,VALUE_COLUMN, UNIT_RATE_COLUMN, QUANTITY_COLUMN, GROSS_WEIGHT_COLUMN,SHIP_COUNT_COLUMN, PORT_COUNT_COLUMN, 'ARIMA_Residuals']  # Adjust as needed
aggregated_df[FEATURE_COLUMNS] = aggregated_df[FEATURE_COLUMNS].fillna(0)
X, y = data_processing.prepare_features_and_target(aggregated_df, FEATURE_COLUMNS, 'spikes')

# Split data (using sklearn's train_test_split)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features
X_train_scaled, X_test_scaled = data_processing.scale_features(X_train_raw, X_test_raw)

# Create sequences
X_train, y_train = data_processing.create_sequences(X_train_scaled, y_train, SPIKES_WINDOW_SIZE)
X_test, y_test = data_processing.create_sequences(X_test_scaled, y_test, SPIKES_WINDOW_SIZE)

# Evaluate all models & save in file
output_file_path = f'{COMMODITY}_results_volza_ais_arima.csv'
models.evaluate_all(X_train, y_train, X_test, y_test, output_file_path)

        Date          Value  Std. Quantity (KG)  Gross Weight   
1 2020-01-02  112499.980000            50000.00       50080.0  \
2 2020-01-03  874407.619695           100172.26       25040.0   
3 2020-01-04  295059.660430           135000.00           0.0   
4 2020-01-05   44200.000000            20000.00           0.0   
5 2020-01-06   17800.239490             5001.00        2218.5   

   Std. Unit Rate ($/KG)  ship_count popular_port  popular_port_count   
1               2.250000         0.0          NaN                 0.0  \
2             768.449153         0.0          NaN                 0.0   
3               2.188610         0.0          NaN                 0.0   
4               2.210000         0.0          NaN                 0.0   
5            1450.844898         0.0          NaN                 0.0   

     Price  Brent Oil Value  WTI Oil Value  spikes  ARIMA_Residuals  
1  14150.0            67.05          61.17       0     14150.000000  
2  14100.0            69.08   

,Name,Params,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1)
0,LSTM,250 layers,0.765714,0.851613,0.880000,0.865574,0.100000,0.08,0.088889
1,LSTM,200 layers,0.857143,0.874251,0.973333,0.921136,0.500000,0.16,0.242424
2,LSTM,100 layers,0.754286,0.849673,0.866667,0.858086,0.090909,0.08,0.085106
3,LSTM,50 layers,0.742857,0.888889,0.800000,0.842105,0.250000,0.40,0.307692
4,CNN with Attention,"32 filters, kernel size 7",0.862857,0.893750,0.953333,0.922581,0.533333,0.32,0.400000
5,CNN with Attention,"32 filters, kernel size 5",0.862857,0.884146,0.966667,0.923567,0.545455,0.24,0.333333
6,CNN with Attention,"32 filters, kernel size 3",0.851429,0.873494,0.966667,0.917722,0.444444,0.16,0.235294
7,CNN with Attention,"64 filters, kernel size 7",0.834286,0.880503,0.933333,0.906149,0.375000,0.24,0.292683
8,CNN with Attention,"64 filters, kernel size 5",0.845714,0.872727,0.960000,0.914286,0.400000,0.16,0.228571
9,CNN with Attention,"64 filters, kernel size 3",0.845714,0.863905,0.973333,0.915361,0.333333,0.08,0.129032
